In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

In [2]:
from google.colab import files
uploaded = files.upload();

Saving binary_dataset.csv to binary_dataset.csv


In [3]:
# 데이터 불러오기 기능(★)
def get_data(file_path : str = 'binary_dataset.csv' ):
    '''
    파일의 경로를 받아 pd.DataFrame타입의 데이터를 반환한다.

    parameter
        - file_path : 파일 경로

    return:
        - pandas.DataFrame타입의 데이터
    '''
    return pd.read_csv(file_path)

#시그모이드 함수
def sigmoid(x):
    '''
    pandas.DataFrame, pandas.Series, numpy.ndarray, int, float 타입을 받아
    해당 값에 시그모이드 함수를 적용시킨 결과를 반환한다.

    paramter
        - x:pandas.DataFrame, pandas.Series, numpy.ndarray, int, float

    return:
        - pandas.DataFrame : 모든 수치형 values에 시그모이드 함수 적용시킨 결과를 반환
        - pandas.Series    : 모든 수치형 values에 시그모이드 함수 적용시킨 결과를 반환
        - numpy.ndarray    : 모든 수치형 원소들에 시그모이드 함수를 적용시킨 결과를 반환
        - int              : 시그모이드 함수를 적용시킨 결과를 반환
        - float            : 시그모이드 함수를 적용시킨 결과를 반환
    '''
    return 1/(1+np.exp(-x))

# 크로스 엔트로피 손실값
def cross_entropy(pred : np.array, target: np.array):
    '''
    예측 확률값과 target값을 numpy.ndarray형태로 받아서 cross_entropy loss를 반환

    paramter:
        - pred   : 예측 확률값이 담긴 numpy.ndarray
        - target : target값이 담긴 numpy.ndarray

    return: 
        - cross_entropy_loss값
    '''
    delta = 1e-7
    return -np.sum(target*np.log(pred+delta))

# 독립 종속 변수 분리
def split_X_y(df: pd.DataFrame , target: str):
    '''
    pandas.DataFrame과 target변수 명을 입력받아서 pandas.DataFrame 형태의 input
    데이터와 pandas.Series형태의 target데이터를 반환한다.

    paramter:
        - df     : 분리할 pandas.DataFrame형태의 데이터
        - target : target변수명 str

    return:
        - X : pandas.DataFrame 형태의 input 데이터
        - y : pandas.Series형태의 target데이터
    
    '''
    X = df.drop(target, axis=1)
    y = df[target]
    return X, y

# 데이터셋 분리 기능(★)
class split_train_test:
    '''
    트레이닝셋과 테스트셋을 분리하는 기능

    Args:
        - *array       : array 하나 이상의 numpy.ndarray, pandas.DataFrame, pandas.Series를 받음
        - shuffle      : boolean 값을 받아 True면 데이터를 섞고 False면 섞지 않음 (default = False)
        - test_szie    : 데스트셋의 크기를 비율로 표시 (default = 0.2)
        - randaom_seed : 난수 고정을 위한 시드값 (default = None)

    change_test_size:
        테스트 사이즈의 비율을 변경한다.
    
        Args:
            - test_szie    : 데스트셋의 크기를 비율로 표시
            - randaom_seed : 난수 고정을 위한 시드값 (default = None)
    
    get_trainset:
        입력 데이터셋에서 트레인셋을 분리하여 반환한다.

        return: 트레인데이터셋

    get_testset:
        입력 데이터셋에서 테스트셋을 분리하여 반환한다.

        return: 테스트 데이터셋
    '''
    def __init__(self, *array, shuffle : bool = False, test_size : float = 0.2, random_seed : int =None):
        if random_seed !=None:
            np.random.seed(random_seed)
        self.array = array
        if shuffle == True:
            self.array= [df.sample(frac=1.0).reset_index(drop=True) for df in self.array]
        self.split_idx = tuple(int(len(df)*(1-test_size)) for df in self.array)
        self.train = None
        self.test = None

    def change_test_size(self, test_size : float , random_seed : int=None):
        if random_seed != None:
            np.random.seed(random_seed)
        self.split_idx = tuple(int(len(df)*(1-test_size)) for df in self.array)
        self.train = None
        self.test = None
    # 학습 데이터 분리 기능(★)
    def get_trainset(self):
        self.train = tuple(df[:idx] for df,idx in tuple(zip(self.array,self.split_idx)))
        if len(self.train) ==1:
            return self.train[0]
        elif len(self.train) >1:
            return self.train
    # 테스트 데이터 분리 기능(★)
    def get_testset(self):
        self.test = tuple(df[idx:] for df,idx in tuple(zip(self.array,self.split_idx)))
        if len(self.test) ==1:
            return self.test[0]
        elif len(self.test) >1:
            return self.test

# 스케일링 기능
class Scaler:
    '''
    pd.DataFrame형태의 데이터를 입력받아 스케일링된 값들을 갖는 
    데이터프레임을 반환한다.

    Args: 
        - method : 스케일러의 유형 선택 ['MinMax', 'Standard']
    
    fit:
        해당 데이터에 맞는 스케일러를 생성한다.
    
        Args: 
            - df : 스케일러 생성 기준이 되는 pandas.DataFrame, pandas.Series, numpy.ndarray형태의 데이터
    
    transform:
        생성된 스케일러를 입력받은 데이터에 적용시킨다.
    
        Args:
            - df : 스케일러를 적용할 pandas.DataFrame, pandas.Series, numpy.ndarray형태의 데이터
    
        return:
            스케일링이 적용된 데이터 프레임
        

    fit_transform:
        fit과 transform 메서드의 기능을 한번에 수행한다.
    
        Args:
            - df : 스케일러를 생성하고 적용시킬 pandas.DataFrame, pandas.Series, numpy.ndarray형태의 데이터
    
        return:
            스케일링이 적용된 데이터 프레임
    
    '''
    def __init__(self, method : str = 'MinMax'):
        self.scaler = None
        self.divide = None
        self.method = method
        self.avg = None 
        self.mini = None
        if method not in ['MinMax', 'Standard']:
            raise ValueError("Select method list : 'MinMax', 'Standard'")

    def fit(self, df ):
        if self.method == 'MinMax':
            self.divide = (df.max() - df.min())
            self.mini = df.min()
        elif self.method == 'Standard':
            self.divide = df.std()
            self.avg = df.mean()

    def transform(self,df):
        if self.method == 'MinMax':
            return (df - self.mini) / self.divide
        elif self.method == 'Standard':
            return (df - self.avg) / self.divide

    def fit_transform(self, df):
        self.fit(df)
        return self.transform(df)

class Dense:
    '''
    단층 신경망을 위해 만든 Dense class
    Dense Layer 생성 및 순전파, 오차계산 시행

    Args :
        - unit            : 출력값의 수
        - input_len     : 입력값의 변수 개수(default = None)
        - activation_func : 적용할 활성함수(function)


    initialization_parameter:
        가중치와 편향을 초기화
    
        Args:
            - method      : 가중치 초기화 방법 ['Xavier', 'He', 'Random']
            - random_seed : 난수 고정 seed값

    Input:
        입력 데이터를 받아서 self.input_len을 수정하는 기능을함

        Args : 
            - X : 입력 데이터를 입력 (pandas.DataFrame, padnas.Series, numpy.ndarray)
 

    data_shuffle:
        데이터를 뒤섞는 기능

        Args:
            - X : input 데이터 (pandas.DataFrame, padnas.Series, numpy.ndarray)
            - y : target 데이터 (pandas.DataFrame, padnas.Series, numpy.ndarray)

        return 
            - X : shuffle한 input 데이터 (numpy.ndarray)
            - y : shuffle한 target 데이터 (numpy.ndarray)
    
    
    get_loss
        numpy.ndarray 타입의 예측 확률값과 target값을 받아서 손실함수를 계산

        Args:
            - pred_proba : numpy.ndarray 타입의 예측 확률
            - target     : numpy.ndarray 타입의 target
            - for_train  : train에 사용되는지 여부 (default = False)

        return:
            - for_train 
                - True  : 평균 loss를 반환
                - Fasle : return하지 않고 손실함수 리스트에 추가


    get_acc
        np.ndarray 타입의 예측 확률값과 target값을 받아서 정확도를 계산

        Args:
            - pred_result : np.ndarray 타입의 예측값 
            - target      : np.ndarray 타입의 target값
            - for_train  : train에 사용되는지 여부 (default = False)

    batch_gen
        batch단위 연산을 위해 df, batch_size를 입력받아 제너레이터를 만들어준다.
        (나누어 떨어지지 않는다면 나머지는 버려준다.)

        Args:
            - df         : 베치크기만큼 뽑아줄 데이터셋 (pandas.DataFrame, padnas.Series, numpy.ndarray)
            - batch_size : 배치의 크기

        yield: 배치사이즈에 맞는 인풋 데이터 (pandas.DataFrame, padnas.Series, numpy.ndarray)

    forward
        입력값을 받아 예측 값 혹은 예측확률을 반환한다.

        Args:
            - X           : 입력 데이터 (pandas.DataFrame, padnas.Series, numpy.ndarray)
            - batch_size  : 배치 사이즈 (default = None)
            - threshold   : 임계치 (default = 0.5)
            - proba       : return값으로 proba를 반환할지 여부 (default = False)
            - random_seed : 난수 고정을 위한 랜덤 시드 값

        return      
            proba 
                - True  : 결과의 확률값으로 반환 (numpy.ndarray)
                - False : 결과값으로 반환 (numpy.ndarray)
        

    fit(beta)
        학습하며(아직 역전파 과정을 구현못해서 손실함수 계산까지의 과정만 진행) 
        가중치를 반환해주고 매 에포크마다 성능과 손실을 반환한다.

    Args:
        - X            : 입력 데이터 (pandas.DataFrame, padnas.Series, numpy.ndarray)
        - y            : Target 데이터 (pandas.DataFrame, padnas.Series, numpy.ndarray)
        - batch_size   : 배치 사이즈
        - epochs       : 학습할 epoch수 (default = 1)
        - threshold    : 임계치 (default = 0.5)
        - random_seed  : 난수 고정을 위한 seed값
    '''

    def __init__(self, unit : int , activation_func , input_len : int = None):
        self.unit = unit
        self.activation_func = activation_func
        self.input_len = input_len
        self.W =None
        self.b =None
        self.batch_loss = np.array([])
        self.batch_acc = np.array([])
    
    # 파라미터와 편향 생성 기능(★)
    def initialization_parameter(self, method :str = 'Xavier', random_seed: int = None):
        if random_seed != None:
            np.random.seed(random_seed)
        input_len = self.input_len
    
        if method == 'Xavier':
            self.W = np.random.normal(0, 1/np.sqrt(input_len+self.unit), (input_len, self.unit))
        elif method == 'He':
            self.W = np.random.normal(0, 1/np.sqrt(input_len), (input_len, self.unit))
        elif method == 'Random':
            self.W = np.random.randn(input_len, self.unit)
        else:
            raise ValueError("Select method list : 'Xavier', 'He', 'Random'")
        
        self.b = np.random.randn(1,self.unit) 

    def Input(self,X):
        self.input_len = X.shape[-1]

    # 데이터 뒤섞기(★)
    def data_shuffle(self,X,y, random_seed = None):
        if type(X) != np.array:
            X = np.array(X)
        if type(y) != np.array:
            y = np.array(y)
        if random_seed !=None:
            np.random.seed(random_seed)
        idx = np.arange(len(X))
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]
        return X,y

    #loss 연산 기능(★)
    def get_loss(self,pred_proba : np.array, target: np.array, for_train : bool =False):
        if for_train == True:
            self.batch_loss = np.append(self.batch_loss,np.mean(cross_entropy(pred_proba, target)))
        else:
            return np.mean(cross_entropy(np.array(pred_proba), target))

    # 정확도 연산 기능(★)
    def get_acc(self,pred_result: np.array, target : np.array, for_train=False):
        if for_train == True:
            self.batch_acc = np.append(self.batch_acc,np.mean(pred_result==target))
        else:
            return np.mean(pred_result==target)
    
    # batch generator
    def batch_gen(self , X, batch_size):    
        X_len = len(X)
        for batch in range(X_len//batch_size):
            yield X[batch*batch_size:batch*batch_size+batch_size]
        
    # 미니배치를 고려한 학습 데이터 기반의 신경망 연산 및 이진 판단 예측 기능(★)
    def forward(self, X, batch_size = None, threshold = 0.5, proba=False,random_seed =None):
        if self.input_len == None:
            self.Input(X)
        
        if type(self.W) == type(None):
            self.initialization_parameter(random_seed = random_seed)

        if random_seed !=None:
            np.random.seed(random_seed)
        
        if type(X) != np.array:
            X = np.array(X).reshape(-1,self.input_len)
        
        
        pred_proba_res = []
        pred_result_res = []

        if batch_size != None:
            # for mini_batch,batch  in self.batch_gen(X, batch_size):
            mini_batch = self.batch_gen(X, batch_size)
            
            while True:
                try:
                    pred_proba = self.activation_func(next(mini_batch)@self.W + self.b)
                    pred_proba_res.append(pred_proba)
                    pred_result = np.where(pred_proba >= threshold, 1, 0)
                    pred_result_res.append(pred_result)
                except:
                    break
        else: 
            pred_proba = self.activation_func(X@self.W + self.b)
            pred_proba_res.append(pred_proba)
            pred_result = np.where(pred_proba > threshold, 1, 0)
            pred_result_res.append(pred_result)
        
        if proba == True:
            return np.array(pred_proba_res)
        else:
            return np.array(pred_result_res)

    def fit(self, X, y, batch_size : int, epochs :int = 1, threshold : float = 0.5, random_seed :int = None): 
        if self.input_len == None:
            self.Input(X)

        if random_seed !=None:
            np.random.seed(random_seed)

        if type(X) != np.array:
            X = np.array(X)
        if type(y) != np.array:
            y = np.array(y)
        
        if type(self.W) == type(None):
            self.initialization_parameter()

        for epoch in range(1,epochs+1):
            np.random.seed(random_seed)

            X,y = self.data_shuffle(X,y)
            
            self.batch_loss = np.array([])
            self.batch_acc = np.array([])

            pred_proba = self.forward(X, batch_size = batch_size, threshold = threshold, proba= True)
            pred_result = self.forward(X, batch_size = batch_size, threshold = threshold, proba= False)
            target = y[:len(pred_proba.reshape(-1))]
            target = target.reshape(pred_proba.shape)
            
            self.get_loss(pred_proba, target, for_train = True)
        
            self.get_acc(pred_result, target, for_train = True)

            total_loss = round(np.mean(self.batch_loss),3)
            total_acc = round(np.mean(self.batch_acc),3)

            print(f'[EPOCH {epoch}] TrainData - Loss = {total_loss}, Accuracy = {total_acc}')
            '''
            역전파
            '''


class Sequential:
    '''
    하나 이상의 Layer를 쌓아서 순차적으로 순전파 과정을 진행하고 마지막 layer에서 결과를 반환한다.

    add
        layer를 sequence(self.layers)에 추가한다. 이때 layer의 이름을 설정가능하고 따로 지정하지 않는다면
        해당 layer객체의 이름을 사용한다.
        
        Args:
            - layer : sequence에 추가할 layer 객체
            - Name  : 해당 layer의 명 (default = None [이 경우 해당 클래스의 이름으로 layer명을 설정한다.])

    data_shuffle:
        데이터를 뒤섞는 기능

        Args:
            - X : input 데이터 (pandas.DataFrame, padnas.Series, numpy.ndarray)
            - y : target 데이터 (pandas.DataFrame, padnas.Series, numpy.ndarray)

        return 
            - X : shuffle한 input 데이터 (numpy.ndarray)
            - y : shuffle한 target 데이터 (numpy.ndarray)
    
    
    get_loss
        numpy.ndarray 타입의 예측 확률값과 target값을 받아서 손실함수를 계산

        Args:
            - pred_proba : numpy.ndarray 타입의 예측 확률
            - target     : numpy.ndarray 타입의 target
            - for_train  : train에 사용되는지 여부 (default = False)

        return:
            - for_train 
                - True  : 평균 loss를 반환
                - Fasle : return하지 않고 손실함수 리스트에 추가


    get_acc
        np.ndarray 타입의 예측 확률값과 target값을 받아서 정확도를 계산

        Args:
            - pred_result : np.ndarray 타입의 예측값 
            - target      : np.ndarray 타입의 target값
            - for_train  : train에 사용되는지 여부 (default = False)

    batch_gen
        batch단위 연산을 위해 df, batch_size를 입력받아 제너레이터를 만들어준다.
        (나누어 떨어지지 않는다면 나머지는 버려준다.)

        Args:
            - df         : 베치크기만큼 뽑아줄 데이터셋 (pandas.DataFrame, padnas.Series, numpy.ndarray)
            - batch_size : 배치의 크기

        yield: 배치사이즈에 맞는 인풋 데이터 (pandas.DataFrame, padnas.Series, numpy.ndarray)

    forward
        입력값을 받아 예측 값 혹은 예측확률을 반환한다.

        Args:
            - X           : 입력 데이터 (pandas.DataFrame, padnas.Series, numpy.ndarray)
            - batch_size  : 배치 사이즈 (default = None)
            - threshold   : 임계치 (default = 0.5)
            - proba       : return값으로 proba를 반환할지 여부 (default = False)
            - random_seed : 난수 고정을 위한 랜덤 시드 값

        return      
            proba 
                - True  : 결과의 확률값으로 반환 (numpy.ndarray)
                - False : 결과값으로 반환 (numpy.ndarray)
        

    fit(beta)
        학습하며(아직 역전파 과정을 구현못해서 손실함수 계산까지의 과정만 진행) 
        가중치를 반환해주고 매 에포크마다 성능과 손실을 반환한다.

    Args:
        - X            : 입력 데이터 (pandas.DataFrame, padnas.Series, numpy.ndarray)
        - y            : Target 데이터 (pandas.DataFrame, padnas.Series, numpy.ndarray)
        - batch_size   : 배치 사이즈
        - epochs       : 학습할 epoch수 (default = 1)
        - threshold    : 임계치 (default = 0.5)
        - random_seed  : 난수 고정을 위한 seed값

    summary
        모델 구조를 출력해주는 메서드
        
        ex)
        ================================================================================
        Layer (type)                    Output Shape                    Params #
        ================================================================================
        Dense_1 (Dense)                  (None,4)                         36
        --------------------------------------------------------------------------------
        Dense_2 (Dense)                  (None,4)                         20
        --------------------------------------------------------------------------------
        Dense_3 (Dense)                  (None,1)                         5
        --------------------------------------------------------------------------------
        ================================================================================
        Total Params :  61
        ================================================================================

    '''
    def __init__(self):
        self.layers = {}
        self.batch_loss = np.array([])
        self.batch_acc = np.array([])
    

    def add(self, layer , Name : str=None):
        if Name == None:
            Name = type(layer).__name__+'_'
            num=1
            Name_select = Name+str(num)
            while Name_select in self.layers.keys():
                num+=1
                Name_select = Name+str(num)
            Name =  Name_select
        
        if  Name in self.layers.keys():
            raise ValueError(f'{Name}은 이미 sequence에 존재합니다.')

        self.layers[Name] = layer

    def data_shuffle(self,X,y, random_seed = None):
        if type(X) != np.array:
            X = np.array(X)
        if type(y) != np.array:
            y = np.array(y)
        if random_seed !=None:
            np.random.seed(random_seed)
        idx = np.arange(len(X))
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]
        return X,y

     #loss 연산 기능(★)
    def get_loss(self,pred_proba, target, for_train=False):
        if for_train == True:
            self.batch_loss = np.append(self.batch_loss,np.mean(cross_entropy(pred_proba, target)))
        else:
            return np.mean(cross_entropy(np.array(pred_proba), target))

    # 정확도 연산 기능(★)
    def get_acc(self,pred_result, target, for_train=False):
        if for_train == True:
            self.batch_acc = np.append(self.batch_acc,np.mean(pred_result==target))
        else:
            return np.mean(pred_result==target)

    def forward(self, X, batch_size :int = None, threshold :float = 0.5, proba :bool =False, random_seed :int =None):
        input = X
        if len(self.layers) > 1:
            for layer in list(self.layers.values())[:-1]:
                input = layer.forward(input, batch_size = batch_size, threshold = threshold, proba = True, random_seed=random_seed)
            result = list(self.layers.values())[-1].forward(input, batch_size = batch_size, threshold = threshold, proba = proba, random_seed=random_seed)
        else:
            result = list(self.layers.values())[-1].forward(input, batch_size = batch_size, threshold = threshold, proba = proba, random_seed=random_seed)

        return result

    
    def fit(self, X, y, batch_size : int =None, epochs :int = 1, threshold :float = 0.5, random_seed : int= None):
        
        for epoch in range(1,epochs+1):
            np.random.seed(random_seed)

            X,y = self.data_shuffle(X,y)
            
            self.batch_loss = np.array([])
            self.batch_acc = np.array([])

            pred_proba = self.forward(X, batch_size = batch_size, threshold = threshold, proba= True)
            pred_result = self.forward(X, batch_size = batch_size, threshold = threshold, proba= False)
            target = y[:len(pred_proba.reshape(-1))]
            target = target.reshape(pred_proba.shape)
            
            self.get_loss(pred_proba, target, for_train = True)
        
            self.get_acc(pred_result, target, for_train = True)

            total_loss = round(np.mean(self.batch_loss),3)
            total_acc = round(np.mean(self.batch_acc),3)

            print(f'[EPOCH {epoch}] TrainData - Loss = {total_loss}, Accuracy = {total_acc}')
    
    def summary(self):
        print('='*80)
        print('Layer (type)'+' '*20+'Output Shape'+ ' '*20+'Params #')
        print('='*80)
        last_name = list(self.layers.keys())[-1]
        total_Params = 0
        for name, layer in self.layers.items():
            layer_type = ' ('+type(layer).__name__+')'
            name += layer_type
            name_len = len(name)
            output_shape = f'(None,{layer.unit})'
            output_shape_len = len(output_shape)
            Params = layer.unit*(layer.input_len+1)
            total_Params += Params
            print(name+' '*(33 - name_len)+ output_shape + ' '*(33-output_shape_len)+ str(Params))
            print('-'*80)
        print('='*80)
        print('Total Params : ', total_Params)
        print('='*80)



In [4]:
def main():
    
    df = get_data()
    
    spliter = split_train_test(df)
    
    train = spliter.get_trainset()
    
    test = spliter.get_testset()

    X_train, y_train = split_X_y(train, 'y')
    X_test, y_test = split_X_y(test, 'y')
    
    scaler = Scaler()

    scaler.fit_transform(X_train)
    scaler.transform(X_test)

    Net = Sequential()

    Net.add(Dense(1,sigmoid))

    Net.fit(X_train, y_train, batch_size=4)



In [5]:
main()

[EPOCH 1] TrainData - Loss = 48.288, Accuracy = 0.75


# 필수 구현 함수(메서드) 테스트

In [6]:
#@title 데이터로드 함수
file_path = "binary_dataset.csv" #@param {type:"string"}
df = get_data(file_path)
df

,x1,x2,x3,x4,x5,x6,x7,x8,y
0,136.093750,51.691005,-0.045909,-0.271816,9.342809,38.096400,4.345438,18.673649,0
1,99.367188,41.572202,1.547197,4.154106,27.555184,61.719016,2.208808,3.662680,1
2,100.890625,51.890394,0.627487,-0.026498,3.883779,23.045267,6.953168,52.279440,0
3,120.554688,45.549905,0.282924,0.419909,1.358696,13.079034,13.312141,212.597029,1
4,121.882812,53.042675,0.200521,-0.282219,2.116221,16.580876,8.947603,91.011762,0
5,125.210938,51.175197,0.139851,-0.385737,1.147993,12.414012,14.068797,228.131554,0
6,141.968750,50.470898,0.244974,-0.342665,2.823579,16.238188,8.207744,85.532584,0
7,136.500000,49.932767,0.044623,-0.374311,1.555184,12.813538,13.314339,214.813089,0
8,83.679688,36.379281,0.572532,2.664611,4.040970,23.169129,7.006681,53.514005,0
9,27.765625,28.666042,5.770087,37.419009,73.112876,62.070220,1.268206,1.082920,1


In [7]:
#@title 트레인 테스트셋 분리

file_path = "binary_dataset.csv" 
df = get_data(file_path)

print('원본 데이터\n')
print(df)
print('-'*80)


shuffle = False #@param {type:"boolean"}
test_size = 0.7 #@param {type:"slider", min:0, max:1, step:0.1}
random_seed = 0 #@param {type:"integer"}
dataset_type = "Test" #@param ["Train", 'Test']


spliter = split_train_test(df, shuffle = shuffle, test_size= test_size, random_seed = random_seed)
train = spliter.get_trainset()    
test = spliter.get_testset()

train_test_dict = {'Train': train,
                   'Test': test}

print(f'''shuffle:{shuffle}
      test_size:{test_size}
      random_seed:{random_seed}      
      ''')

print(f'''dataset_type : {dataset_type }
      
      {train_test_dict[dataset_type]}
      ''')




원본 데이터

            x1         x2        x3         x4          x5         x6  \
0   136.093750  51.691005 -0.045909  -0.271816    9.342809  38.096400   
1    99.367188  41.572202  1.547197   4.154106   27.555184  61.719016   
2   100.890625  51.890394  0.627487  -0.026498    3.883779  23.045267   
3   120.554688  45.549905  0.282924   0.419909    1.358696  13.079034   
4   121.882812  53.042675  0.200521  -0.282219    2.116221  16.580876   
5   125.210938  51.175197  0.139851  -0.385737    1.147993  12.414012   
6   141.968750  50.470898  0.244974  -0.342665    2.823579  16.238188   
7   136.500000  49.932767  0.044623  -0.374311    1.555184  12.813538   
8    83.679688  36.379281  0.572532   2.664611    4.040970  23.169129   
9    27.765625  28.666042  5.770087  37.419009   73.112876  62.070220   
10  135.859375  51.937272  0.065769  -0.366114   20.774247  52.772648   
11  114.281250  41.253965  0.411821   0.616996    2.412207  20.427942   
12  112.437500  38.295673  0.501943   1.074

In [8]:
#@title 파라미터와 평향 생성기능
unit = 1 #@param {type:"integer"}
input_len = 8 #@param {type:"integer"}
activation_func = sigmoid

Net = Dense(unit = unit, activation_func = activation_func, input_len = input_len)

method = 'Xavier' #@param ['Xavier', 'He', 'Random']
random_seed = 0 #@param {type:"integer"}


Net.initialization_parameter(method = method, random_seed = random_seed)

print(f'''W
{Net.W}
''')
print('-'*80)
print(f'''b
{Net.b}
''')


W
[[ 0.58801745]
 [ 0.13338574]
 [ 0.32624599]
 [ 0.7469644 ]
 [ 0.62251933]
 [-0.32575929]
 [ 0.31669614]
 [-0.0504524 ]]

--------------------------------------------------------------------------------
b
[[-0.10321885]]



In [9]:
#@title # 데이터 뒤섞기

df = get_data()
    
X, y = split_X_y(df, 'y')

X = np.array(X)
y = np.array(y)

Net = Dense(1, sigmoid)

ranodm_seed = 0 #@param {type:"integer"}

shuffle_X, shuffle_y= Net.data_shuffle(X, y, random_seed = random_seed)

raw_dict = {'X': X, 'y': y}

shuffle_dict = {'X': shuffle_X, 'y': shuffle_y}

select = "y" #@param ["X", "y"] 

print(f'''
raw_{select}
{raw_dict[select]}

''')
print('-'*80)

print(f'''
shuffled_{select}
{shuffle_dict[select]}
''')






raw_y
[0 1 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0]


--------------------------------------------------------------------------------

shuffled_y
[0 1 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 0]



In [10]:
#@title 손심 함수 계산
df = get_data()
    
X, y = split_X_y(df, 'y')

X = np.array(X)
y = np.array(y)

Net = Dense(1, sigmoid)

pred_proba = Net.forward(X, proba=True)

print('cross_entropy: ',Net.get_loss(pred_proba, y))

cross_entropy:  -1.1952977317321753e-05


In [11]:
#@title 정확도 계산
df = get_data()
    
X, y = split_X_y(df, 'y')

X = np.array(X)
y = np.array(y)

Net = Dense(1, sigmoid)

pred_result = Net.forward(X)

print('accuracy : ', Net.get_acc(pred_result, y))

accuracy :  0.66


In [12]:
#@title 배치 단위 결과 반환

df = get_data()
    
X, y = split_X_y(df, 'y')

X = np.array(X)

Net = Dense(1,sigmoid)

batch_size = 6 #@param {type:"integer"}
threshold = 0.5 #@param {type:"slider", min:0, max:1, step:0.1}
proba = False #@param {type:"boolean"}
random_seed = 0 #@param {type:"integer"}




Net.forward(X, batch_size = batch_size, threshold = threshold, proba=proba,random_seed =None)

array([[[1],
        [1],
        [1],
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        [0],
        [1],
        [1]],

       [[1],
        [0],
        [1],
        [1],
        [1],
        [1]]])

# 추가 구현 클래스(Sequential) 테스트

In [13]:
#@title 다층 신경망 순전파 + 오차,score 계산

df = get_data()

spliter = split_train_test(df)

train = spliter.get_trainset()

test = spliter.get_testset()

X_train, y_train = split_X_y(train, 'y')
X_test, y_test = split_X_y(test, 'y')

scaler = Scaler()

scaler.fit_transform(X_train)
scaler.transform(X_test)

Net = Sequential()

hidden_layer_num = 2 #@param {type:"integer"}

# 히든 노드의 아웃풋 크기를 순차적으로 리스트 형태로 입력
hidden_node = [4,4] #@param {type:"raw"}

for i in range(hidden_layer_num):
    Net.add(Dense(hidden_node[i],sigmoid))

Net.add(Dense(1,sigmoid))

Net.fit(X_train, y_train, batch_size=4)

print('\n')
print('Model summary')
Net.summary()

[EPOCH 1] TrainData - Loss = 6.328, Accuracy = 0.688


Model summary
Layer (type)                    Output Shape                    Params #
Dense_1 (Dense)                  (None,4)                         36
--------------------------------------------------------------------------------
Dense_2 (Dense)                  (None,4)                         20
--------------------------------------------------------------------------------
Dense_3 (Dense)                  (None,1)                         5
--------------------------------------------------------------------------------
Total Params :  61
